In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

##定義ResNet模型

In [ ]:
def resnet_block(inputs, filters, kernel_size=3, stride=1, activation='relu'):
    x = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    if activation:
        x = layers.Activation(activation)(x)
    return x

def resnet_v1(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = resnet_block(inputs, 64)
    x = resnet_block(x, 64)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = resnet_block(x, 128, stride=2)
    x = resnet_block(x, 128)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = resnet_block(x, 256, stride=2)
    x = resnet_block(x, 256)
    x = layers.MaxPooling2D(pool_size=2)(x)

    x = resnet_block(x, 512, stride=2)
    x = resnet_block(x, 512)
    x = layers.GlobalAveragePooling2D()(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

# 定義ResNet模型(詳細)

In [ ]:
class BasicBlock(tf.keras.Model):
    def __init__(self, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=3, strides=stride, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, strides=1, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.downsample = models.Sequential()
        if stride != 1:
            self.downsample.add(layers.Conv2D(out_channels, kernel_size=1, strides=stride, use_bias=False))
            self.downsample.add(layers.BatchNormalization())

    def call(self, x, training=False):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out, training=training)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out, training=training)
        if self.downsample.layers:
            identity = self.downsample(identity, training=training)
        out += identity
        out = self.relu(out)
        return out

class ResNet(tf.keras.Model):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = layers.Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False, input_shape=(28, 28, 1))
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.maxpool = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = [block(out_channels, stride)]
        for _ in range(1, blocks):
            layers.append(block(out_channels))
        return models.Sequential(layers)

    def call(self, x, training=False):
        x = self.conv1(x)
        x = self.bn1(x, training=training)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        x = self.fc(x)
        return x

# 創建ResNet-18模型
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

# Device configuration
device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

with tf.device(device):
    model = ResNet18()

#創建ResNet模型

In [ ]:
model = resnet_v1(input_shape=(28, 28, 1), num_classes=10)

#編譯模型

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#加載MNIST數據

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

#訓練模型

In [ ]:
model.fit(train_images, train_labels, epochs=10, batch_size=64, validation_data=(test_images, test_labels))

#評估模型

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Accuracy: {test_acc * 100}%')